In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init()

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext
lines = sc.textFile("spark-2.4.4-bin-hadoop2.7/README.md")

In [0]:
import os
import sqlite3

In [0]:
from pyspark.sql import HiveContext, Row
# Or if you can't include the hive requirements 
from pyspark.sql import SQLContext, Row

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [0]:
import pandas as pd
cnx = sqlite3.connect(r'/content/drive/My Drive/database.sqlite')

In [0]:
country = pd.read_sql_query("SELECT * FROM Country", cnx)
league = pd.read_sql_query("SELECT * FROM League", cnx)
match = pd.read_sql_query("SELECT * FROM Match", cnx)
player = pd.read_sql_query("SELECT * FROM Player", cnx)
player_attributes = pd.read_sql_query("SELECT * FROM Player_attributes", cnx)
team = pd.read_sql_query("SELECT * FROM Team", cnx)
team_attributes = pd.read_sql_query("SELECT * FROM Team_attributes", cnx)

In [0]:
sdf_country = spark.createDataFrame(country)
sdf_league = spark.createDataFrame(league)
sdf_match = spark.createDataFrame(match)
sdf_player = spark.createDataFrame(player)
sdf_player_attributes = spark.createDataFrame(player)
sdf_team = spark.createDataFrame(team)
sdf_team_attributes = spark.createDataFrame(team_attributes)


In [0]:
sdf_player.createOrReplaceTempView("player")

sdf_country.createOrReplaceTempView("country")

sdf_league.createOrReplaceTempView("league")
sdf_match.createOrReplaceTempView("match")

sdf_player_attributes.createOrReplaceTempView("Player_attributes")
sdf_team.createOrReplaceTempView("team")
sdf_team_attributes.createOrReplaceTempView("team_attributes")

In [11]:
sdf_player.printSchema()

sdf_country.printSchema()

root
 |-- id: long (nullable = true)
 |-- player_api_id: long (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_fifa_api_id: long (nullable = true)
 |-- birthday: string (nullable = true)
 |-- height: double (nullable = true)
 |-- weight: long (nullable = true)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)



Answer to Question1 -

In [15]:
sqlDF = spark.sql("SELECT player_name, birthday FROM player WHERE birthday between '1987' and '1991' ORDER BY birthday")
sqlDF.show()

+--------------------+-------------------+
|         player_name|           birthday|
+--------------------+-------------------+
|       Abdullah Omar|1987-01-01 00:00:00|
|         Ben Alnwick|1987-01-01 00:00:00|
|           Loic Remy|1987-01-02 00:00:00|
|       Robert Milsom|1987-01-02 00:00:00|
|           Celestino|1987-01-02 00:00:00|
|           Luis Dias|1987-01-03 00:00:00|
|        Chris Turner|1987-01-03 00:00:00|
|        Dani Estrada|1987-01-03 00:00:00|
|    Przemyslaw Tyton|1987-01-04 00:00:00|
|           Kay Voser|1987-01-04 00:00:00|
|       Hans Martinez|1987-01-04 00:00:00|
|       Danny Simpson|1987-01-04 00:00:00|
|        Migjen Basha|1987-01-05 00:00:00|
|   Esteban Casagolda|1987-01-05 00:00:00|
|Claudio Lustenberger|1987-01-06 00:00:00|
|       Davide Astori|1987-01-07 00:00:00|
|  Michael McGlinchey|1987-01-07 00:00:00|
|      Stefan Babovic|1987-01-07 00:00:00|
|         Lucas Leiva|1987-01-09 00:00:00|
|     Michele Rinaldi|1987-01-09 00:00:00|
+----------

Answer to Question 2 -

In [16]:
sqlDF = spark.sql("SELECT country.name, league.name, sum(match.home_team_goal+match.away_team_goal) from country inner join league on country.id = league.country_id join match on league.id = match.league_id group by league.name order by sum(match.home_team_goal+match.away_team_goal) desc")
sqlDF.show()

AnalysisException: ignored